In [ ]:
# -*- coding: utf-8 -*-
"""
Interactive Data Gating GUI Application
======================================
一个用于交互式数据选择和可视化的GUI应用程序
"""

import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import threading
import queue
import os
from openflow import InteractivePolygonGating, InteractiveHistogramThreshold

# 导入您提供的自定义类
# 注意：假设这些类在一个叫interactive_gating.py的文件中
# from interactive_gating import InteractivePolygonGating, InteractiveHistogramThreshold

class DataGatingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("交互式数据选择分析工具")
        self.root.geometry("1000x700")
        
        # 实例变量初始化
        self.df = None
        self.polygon_gate = None
        self.histogram_gate = None
        self.selected_data = None
        
        # 创建UI组件
        self.create_widgets()
        
        # 消息队列，用于从matplotlib线程传递消息到主GUI线程
        self.message_queue = queue.Queue()
        self.root.after(100, self.check_message_queue)
    
    def create_widgets(self):
        """创建所有GUI组件"""
        # 主框架分为左侧控制区和右侧图形区
        self.main_frame = ttk.PanedWindow(self.root, orient=tk.HORIZONTAL)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # 左侧控制面板
        self.control_frame = ttk.Frame(self.main_frame, width=300)
        self.main_frame.add(self.control_frame, weight=1)
        
        # 右侧图形显示区
        self.plot_frame = ttk.Frame(self.main_frame)
        self.main_frame.add(self.plot_frame, weight=3)
        
        # 设置控制面板中的组件
        self.setup_control_panel()
        
        # 设置图形显示区
        self.setup_plot_area()
    
    def setup_control_panel(self):
        """设置左侧控制面板"""
        # 数据加载区
        data_frame = ttk.LabelFrame(self.control_frame, text="数据加载")
        data_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Button(data_frame, text="加载CSV文件", command=self.load_data).pack(fill=tk.X, padx=5, pady=5)
        ttk.Button(data_frame, text="生成示例数据", command=self.generate_sample_data).pack(fill=tk.X, padx=5, pady=5)
        
        self.data_info_label = ttk.Label(data_frame, text="未加载数据")
        self.data_info_label.pack(fill=tk.X, padx=5, pady=5)
        
        # 多边形选择区
        polygon_frame = ttk.LabelFrame(self.control_frame, text="多边形选择")
        polygon_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # X轴列选择
        ttk.Label(polygon_frame, text="X轴列:").pack(anchor=tk.W, padx=5, pady=2)
        self.x_col_var = tk.StringVar()
        self.x_col_combo = ttk.Combobox(polygon_frame, textvariable=self.x_col_var, state="readonly")
        self.x_col_combo.pack(fill=tk.X, padx=5, pady=2)
        
        # Y轴列选择
        ttk.Label(polygon_frame, text="Y轴列:").pack(anchor=tk.W, padx=5, pady=2)
        self.y_col_var = tk.StringVar()
        self.y_col_combo = ttk.Combobox(polygon_frame, textvariable=self.y_col_var, state="readonly")
        self.y_col_combo.pack(fill=tk.X, padx=5, pady=2)
        
        # 顶点数量
        ttk.Label(polygon_frame, text="多边形顶点数:").pack(anchor=tk.W, padx=5, pady=2)
        self.vertex_var = tk.IntVar(value=5)
        vertex_spin = ttk.Spinbox(polygon_frame, from_=3, to=10, textvariable=self.vertex_var)
        vertex_spin.pack(fill=tk.X, padx=5, pady=2)
        
        # 对数刻度选择
        self.log_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(polygon_frame, text="使用对数刻度", variable=self.log_var).pack(anchor=tk.W, padx=5, pady=5)
        
        # 启动多边形选择按钮
        ttk.Button(polygon_frame, text="启动多边形选择", command=self.start_polygon_gating).pack(fill=tk.X, padx=5, pady=5)
        
        # 直方图阈值选择区
        histogram_frame = ttk.LabelFrame(self.control_frame, text="直方图阈值选择")
        histogram_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # 阈值通道选择
        ttk.Label(histogram_frame, text="阈值通道:").pack(anchor=tk.W, padx=5, pady=2)
        self.threshold_var = tk.StringVar()
        self.threshold_combo = ttk.Combobox(histogram_frame, textvariable=self.threshold_var, state="readonly")
        self.threshold_combo.pack(fill=tk.X, padx=5, pady=2)
        
        # 可视化通道选择（多选）
        ttk.Label(histogram_frame, text="可视化通道:").pack(anchor=tk.W, padx=5, pady=2)
        self.plot_channels_frame = ttk.Frame(histogram_frame)
        self.plot_channels_frame.pack(fill=tk.X, padx=5, pady=2)
        self.plot_channel_vars = {}  # 将在加载数据后填充
        
        # 启动直方图阈值选择按钮
        ttk.Button(histogram_frame, text="启动直方图阈值选择", command=self.start_histogram_gating).pack(fill=tk.X, padx=5, pady=5)
        
        # 结果操作区
        result_frame = ttk.LabelFrame(self.control_frame, text="结果操作")
        result_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.result_info_label = ttk.Label(result_frame, text="未选择数据")
        self.result_info_label.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Button(result_frame, text="保存选中数据", command=self.save_selected_data).pack(fill=tk.X, padx=5, pady=5)
    
    def setup_plot_area(self):
        """设置右侧图形显示区"""
        # 在matplotlib图形上方添加标签
        self.plot_label = ttk.Label(self.plot_frame, text="数据可视化区域")
        self.plot_label.pack(fill=tk.X, padx=5, pady=5)
        
        # 创建matplotlib图形和画布
        self.fig = Figure(figsize=(8, 6), dpi=100)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.plot_frame)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # 初始化空图形
        self.ax = self.fig.add_subplot(111)
        self.ax.set_title("未加载数据")
        self.ax.set_xlabel("X")
        self.ax.set_ylabel("Y")
        self.canvas.draw()
    
    def load_data(self):
        """从CSV文件加载数据"""
        file_path = filedialog.askopenfilename(
            title="选择CSV文件",
            filetypes=[("CSV文件", "*.csv"), ("所有文件", "*.*")]
        )
        
        if file_path:
            try:
                self.df = pd.read_csv(file_path)
                self.update_data_info()
                self.update_column_selectors()
                self.preview_data()
                messagebox.showinfo("成功", f"已加载数据，共{len(self.df)}行，{len(self.df.columns)}列。")
            except Exception as e:
                messagebox.showerror("错误", f"加载数据时出错：{str(e)}")
    
    def generate_sample_data(self):
        """生成模拟流式细胞仪数据用于演示"""
        try:
            np.random.seed(42)
            n_samples = 50000
            
            # 创建两个高斯分布的细胞群体
            population1 = {
                'FSC-A': np.random.normal(200, 30, int(n_samples * 0.6)),
                'SSC-A': np.random.normal(150, 25, int(n_samples * 0.6)),
                'FL1-A': np.random.normal(100, 15, int(n_samples * 0.6)),
                'FL2-A': np.random.normal(50, 10, int(n_samples * 0.6)),
                'FL5-A': np.random.normal(80, 12, int(n_samples * 0.6))
            }
            
            population2 = {
                'FSC-A': np.random.normal(250, 35, int(n_samples * 0.4)),
                'SSC-A': np.random.normal(200, 30, int(n_samples * 0.4)),
                'FL1-A': np.random.normal(150, 20, int(n_samples * 0.4)),
                'FL2-A': np.random.normal(70, 15, int(n_samples * 0.4)),
                'FL5-A': np.random.normal(120, 18, int(n_samples * 0.4))
            }
            
            # 合并两个群体
            data = {}
            for channel in population1.keys():
                data[channel] = np.concatenate([population1[channel], population2[channel]])
            
            self.df = pd.DataFrame(data)
            self.update_data_info()
            self.update_column_selectors()
            self.preview_data()
            messagebox.showinfo("成功", "已生成示例数据，模拟两个细胞群体。")
        except Exception as e:
            messagebox.showerror("错误", f"生成数据时出错：{str(e)}")
    
    def update_data_info(self):
        """更新数据信息标签"""
        if self.df is not None:
            info_text = f"数据大小: {len(self.df)}行 × {len(self.df.columns)}列"
            self.data_info_label.config(text=info_text)
    
    def update_column_selectors(self):
        """更新列选择下拉框"""
        if self.df is not None:
            columns = list(self.df.columns)
            
            # 更新X轴和Y轴列选择器
            self.x_col_combo['values'] = columns
            self.y_col_combo['values'] = columns
            if len(columns) > 0:
                self.x_col_var.set(columns[0])
            if len(columns) > 1:
                self.y_col_var.set(columns[1])
            
            # 更新阈值通道选择器
            self.threshold_combo['values'] = columns
            if len(columns) > 0:
                self.threshold_var.set(columns[0])
            
            # 更新可视化通道选择器
            for widget in self.plot_channels_frame.winfo_children():
                widget.destroy()
            
            self.plot_channel_vars = {}
            for col in columns:
                var = tk.BooleanVar(value=True)
                self.plot_channel_vars[col] = var
                chk = ttk.Checkbutton(self.plot_channels_frame, text=col, variable=var)
                chk.pack(anchor=tk.W)
    
    def preview_data(self):
        """在图形区域预览数据"""
        if self.df is not None:
            self.ax.clear()
            
            x_col = self.x_col_var.get()
            y_col = self.y_col_var.get()
            
            if x_col in self.df.columns and y_col in self.df.columns:
                # 随机抽样以加快显示速度
                sample_size = min(5000, len(self.df))
                sample_df = self.df.sample(sample_size)
                
                self.ax.scatter(sample_df[x_col], sample_df[y_col], s=0.5, alpha=0.5)
                self.ax.set_title(f"{x_col} vs {y_col} (随机抽样 {sample_size} 点)")
                self.ax.set_xlabel(x_col)
                self.ax.set_ylabel(y_col)
            else:
                self.ax.set_title("未选择有效的X轴和Y轴列")
            
            self.canvas.draw()
    
    def start_polygon_gating(self):
        """在新线程中启动多边形选择工具"""
        if self.df is None:
            messagebox.showwarning("警告", "请先加载或生成数据。")
            return
        
        x_col = self.x_col_var.get()
        y_col = self.y_col_var.get()
        num_edges = self.vertex_var.get()
        use_log = self.log_var.get()
        
        # 在新线程中运行matplotlib交互
        threading.Thread(
            target=self._run_polygon_gating, 
            args=(x_col, y_col, num_edges, use_log),
            daemon=True
        ).start()
    
    def _run_polygon_gating(self, x_col, y_col, num_edges, use_log):
        """在线程中运行多边形选择"""
        try:
            # 创建多边形选择实例
            self.polygon_gate = InteractivePolygonGating(
                dataframe=self.df,
                x_col=x_col,
                y_col=y_col,
                num_edges=num_edges,
                log=use_log
            )
            
            # 等待matplotlib窗口关闭
            plt.show(block=True)
            
            # 窗口关闭后，获取选中的数据
            self.selected_data = self.polygon_gate.selected_data
            
            # 向主线程发送消息
            if self.selected_data is not None:
                self.message_queue.put({
                    'type': 'polygon_complete',
                    'count': len(self.selected_data)
                })
            else:
                self.message_queue.put({
                    'type': 'polygon_complete',
                    'count': 0
                })
        except Exception as e:
            self.message_queue.put({
                'type': 'error',
                'message': str(e)
            })
    
    def start_histogram_gating(self):
        """在新线程中启动直方图阈值选择工具"""
        if self.selected_data is None:
            messagebox.showwarning("警告", "请先使用多边形选择工具选择数据。")
            return
        
        threshold_channel = self.threshold_var.get()
        plot_channels = [col for col, var in self.plot_channel_vars.items() if var.get()]
        
        if not plot_channels:
            messagebox.showwarning("警告", "请至少选择一个可视化通道。")
            return
        
        # 在新线程中运行matplotlib交互
        threading.Thread(
            target=self._run_histogram_gating, 
            args=(threshold_channel, plot_channels),
            daemon=True
        ).start()
    
    def _run_histogram_gating(self, threshold_channel, plot_channels):
        """在线程中运行直方图阈值选择"""
        try:
            # 创建直方图阈值选择实例
            self.histogram_gate = InteractiveHistogramThreshold(
                data=self.selected_data,
                threshold_channel=threshold_channel,
                plot_channels=plot_channels
            )
            
            # 等待matplotlib窗口关闭
            plt.show(block=True)
            
            # 窗口关闭后，获取最终选中的数据
            final_data = self.histogram_gate.selected_data
            
            # 如果有效，更新选中数据
            if final_data is not None:
                self.selected_data = final_data
                
                # 向主线程发送消息
                self.message_queue.put({
                    'type': 'histogram_complete',
                    'count': len(self.selected_data)
                })
            else:
                self.message_queue.put({
                    'type': 'histogram_complete',
                    'count': 0
                })
        except Exception as e:
            self.message_queue.put({
                'type': 'error',
                'message': str(e)
            })
    
    def save_selected_data(self):
        """保存选中的数据到CSV文件"""
        if self.selected_data is None or len(self.selected_data) == 0:
            messagebox.showwarning("警告", "没有选中的数据可保存。")
            return
        
        file_path = filedialog.asksaveasfilename(
            title="保存选中数据",
            defaultextension=".csv",
            filetypes=[("CSV文件", "*.csv"), ("所有文件", "*.*")]
        )
        
        if file_path:
            try:
                self.selected_data.to_csv(file_path, index=False)
                messagebox.showinfo("成功", f"已将{len(self.selected_data)}行数据保存到 {os.path.basename(file_path)}。")
            except Exception as e:
                messagebox.showerror("错误", f"保存数据时出错：{str(e)}")
    
    def check_message_queue(self):
        """检查消息队列，处理从其他线程发来的消息"""
        try:
            while True:
                message = self.message_queue.get_nowait()
                
                if message['type'] == 'polygon_complete':
                    count = message['count']
                    if count > 0:
                        self.result_info_label.config(text=f"多边形选择：{count}行数据")
                        # 更新预览图
                        self.update_selected_preview()
                    else:
                        self.result_info_label.config(text="多边形选择：无数据选中")
                
                elif message['type'] == 'histogram_complete':
                    count = message['count']
                    if count > 0:
                        self.result_info_label.config(text=f"最终选择：{count}行数据")
                        # 更新预览图
                        self.update_selected_preview()
                    else:
                        self.result_info_label.config(text="直方图阈值选择：无数据选中")
                
                elif message['type'] == 'error':
                    messagebox.showerror("错误", message['message'])
        
        except queue.Empty:
            pass
        
        # 每100毫秒检查一次消息队列
        self.root.after(100, self.check_message_queue)
    
    def update_selected_preview(self):
        """更新图形区域显示选中的数据"""
        if self.selected_data is not None and len(self.selected_data) > 0:
            self.ax.clear()
            
            # 获取X和Y轴列名
            x_col = self.x_col_var.get()
            y_col = self.y_col_var.get()
            
            if x_col in self.selected_data.columns and y_col in self.selected_data.columns:
                # 显示原始数据作为背景（灰色）
                if self.df is not None:
                    sample_size = min(5000, len(self.df))
                    sample_df = self.df.sample(sample_size)
                    self.ax.scatter(sample_df[x_col], sample_df[y_col], s=0.5, alpha=0.1, color='gray')
                
                # 显示选中的数据（红色）
                self.ax.scatter(self.selected_data[x_col], self.selected_data[y_col], 
                               s=1, alpha=0.5, color='red')
                
                self.ax.set_title(f"选中数据: {x_col} vs {y_col} ({len(self.selected_data)}点)")
                self.ax.set_xlabel(x_col)
                self.ax.set_ylabel(y_col)
            else:
                self.ax.set_title("无法显示选中数据：列名不匹配")
            
            self.canvas.draw()


if __name__ == "__main__":
    root = tk.Tk()
    app = DataGatingApp(root)
    root.mainloop()

/var/folders/5m/_4v3s_255xn0mk45gz3c5_8r0000gr/T/ipykernel_6531/1763037332.py:148: UserWarning: Glyph 26410 (\N{CJK UNIFIED IDEOGRAPH-672A}) missing from font(s) DejaVu Sans.
  self.canvas.draw()
/var/folders/5m/_4v3s_255xn0mk45gz3c5_8r0000gr/T/ipykernel_6531/1763037332.py:148: UserWarning: Glyph 21152 (\N{CJK UNIFIED IDEOGRAPH-52A0}) missing from font(s) DejaVu Sans.
  self.canvas.draw()
/var/folders/5m/_4v3s_255xn0mk45gz3c5_8r0000gr/T/ipykernel_6531/1763037332.py:148: UserWarning: Glyph 36733 (\N{CJK UNIFIED IDEOGRAPH-8F7D}) missing from font(s) DejaVu Sans.
  self.canvas.draw()
/var/folders/5m/_4v3s_255xn0mk45gz3c5_8r0000gr/T/ipykernel_6531/1763037332.py:148: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.
  self.canvas.draw()
/var/folders/5m/_4v3s_255xn0mk45gz3c5_8r0000gr/T/ipykernel_6531/1763037332.py:148: UserWarning: Glyph 25454 (\N{CJK UNIFIED IDEOGRAPH-636E}) missing from font(s) DejaVu Sans.
  self.canvas.draw()
invalid command name